In [16]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

from lightgbm import LGBMClassifier
from itertools import chain, combinations

In [17]:
import pandas as pd
import os 
import re
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.datasets import make_classification

import os
from collections import defaultdict

In [18]:
fr_groundtruths =['USER_FR', 'URLa_FR', 'URLb_FR']
lr_groundtruths =['HASHTAG','URL_LR','POLITICIAN_1H_LR', 'PARTY_FOLLOWER_LR']

In [19]:
performance = {dataset : defaultdict(dict) for dataset in datasets}
gt_f1 = {dataset : defaultdict(dict) or dataset in dataset}

In [20]:
import itertools
def all_combinations(l):
    return(itertools.product(l, repeat=2))

In [21]:
granularity = '_per_user'
feature = 'use'
dataset='qanda'
datasets = [dataset]
for dataset in datasets:
    for train, test in itertools.chain(all_combinations(lr_groundtruths),all_combinations(fr_groundtruths)):
        print(dataset, train, test)
        train_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+train+'_'+granularity+'.pk')
        if not os.path.exists(train_path):
            continue
        test_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+test+'_'+granularity+'.pk')
        if not os.path.exists(test_path):
            continue
    #         print(train, test)
        def load_vector(fp):
            with open(fp, 'rb') as rf:
                vec = pk.load(rf)
            return vec

        y_train = load_vector(train_path).values
        mask_train = y_train >= 0

        y_test = load_vector(test_path).values
        mask_test = y_test >= 0
        mask = mask_train | mask_test
        if train == 'POLITICIAN_LR' and test == 'POLITICIAN_1H_LR':
            mask = mask_train & mask_test
        if any(mask):
            gt_f1[dataset][train][test] = f1_score(y_train[mask], y_test[mask],average='macro')

            y = y_train[mask]
            feature_path = os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk')
            X_orig = load_vector(feature_path)
            X = X_orig[mask]
            if train == test:
                et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
                # validation instance
                skf = StratifiedKFold(n_splits=5, shuffle=True)
                # getting the model validation predictions
                preds = cross_val_predict(et, X, y, cv=skf, method='predict_proba')
            else:
                et = LGBMClassifier(n_estimators=100, min_data_in_leaf=500,colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
                et.fit(X,y)
                preds = et.predict_proba(X)
            if train in fr_groundtruths:
                preds = preds[:,1]
            # evaluating the model
            print(dataset, train, test)
            try:
                if test == 'POLITICIAN_LR' and train == 'POLITICIAN_1H_LR':
                    N = preds.shape[0]
    #                     print(et.classes_)
                    preds = np.c_[np.zeros(N),preds]

                print('Area under the ROC Curve:', roc_auc_score(y_test[mask], preds, multi_class='ovo'))
                performance[dataset][train][test] = roc_auc_score(y_test[mask], preds, multi_class='ovo')
            except Exception as e:
                print('Failed for:', train, test)
                raise(e)

qanda HASHTAG HASHTAG
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda HASHTAG HASHTAG
Area under the ROC Curve: 0.8820916735160584
qanda HASHTAG URL_LR
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda HASHTAG URL_LR
Area under the ROC Curve: 0.5525303540102299
qanda HASHTAG POLITICIAN_1H_LR
[LightGBM] [Warning] min_data_in_leaf is set=500,

/data/roram/anaconda3/envs/rr/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda POLITICIAN_1H_LR POLITICIAN_1H_LR
Area under the ROC Curve: 0.5398686424577973
qanda POLITICIAN_1H_LR PARTY_FOLLOWER_LR
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda POLITICIAN_1H_LR PARTY_FOLLOWER_LR
Area under the ROC Curve: 0.3911170335797451
qanda PARTY_FOLLOWER_LR HASHTAG
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
qanda PARTY_

In [41]:
pd.DataFrame(dict(performance)).reindex(index=col_order, columns=col_order)

,HASHTAG,URL_LR,POLITICIAN_1H_LR,PARTY_FOLLOWER_LR,USER_FR,URLa_FR,URLb_FR
HASHTAG,0.882092,0.458842,0.385137,0.519819,NaN,NaN,NaN
URL_LR,0.552530,0.820801,0.367717,0.361625,NaN,NaN,NaN
POLITICIAN_1H_LR,0.429365,0.315827,0.539869,0.357079,NaN,NaN,NaN
PARTY_FOLLOWER_LR,0.497467,0.366081,0.391117,0.627986,NaN,NaN,NaN
USER_FR,NaN,NaN,NaN,NaN,0.832443,0.709842,0.777243
URLa_FR,NaN,NaN,NaN,NaN,0.915112,0.886598,0.988088
URLb_FR,NaN,NaN,NaN,NaN,0.874810,0.846904,0.921448


In [44]:
col_order = ['HASHTAG','URL_LR','POLITICIAN_1H_LR','PARTY_FOLLOWER_LR','USER_FR','URLa_FR','URLb_FR']
performance_dfs = {dataset: pd.DataFrame(dict(performance[dataset])).reindex(index=col_order, columns=col_order) for dataset in datasets}

In [26]:
# gt_f1_dfs = {dataset: pd.DataFrame(gt_f1[dataset]).reindex(index=col_order, columns=col_order) for dataset in datasets}

In [53]:
# for dataset in datasets:
#     performance_dfs[dataset].to_csv('./'+dataset+'_cross_gt_auc.csv')
# for dataset in datasets:
#     gt_f1_dfs[dataset].to_csv('./'+dataset+'_cross_gt_f1.csv')

In [46]:
performance_dfs['qanda'].to_csv(os.path.join(base_dir, 'data', '04_results', 'qanda_cross_gt_auc.csv'))